## Assembly AI

### Setup

In [1]:
%%capture
!pip install fireworks-ai
!pip install python-dotenv

In [2]:
%%capture
!pip install assemblyai

In [3]:
import os
from requests.auth import HTTPBasicAuth

In [4]:
import getpass
assemblyai_api_key = getpass.getpass("Enter your API Key...")

In [5]:
import getpass
api_key = getpass.getpass("Enter your API Key...")

In [6]:
import os
os.environ['FIREWORKS_API_KEY'] = api_key

In [7]:
import fireworks.client
fireworks.client.api_key = api_key

In [8]:
%%capture
#openai
!pip install requests
!pip install fireworks-ai
!pip install pytube

In [9]:
import os
import requests
import fireworks.client
import subprocess
from fireworks.client import Fireworks
from pytube import YouTube

## Llama-3

In [10]:
%%capture
! pip install ffmpeg-python pytube

In [11]:
from pytube import YouTube
import subprocess
import requests

In [14]:
import assemblyai as aai
import requests
import time
import os
import subprocess
from pytube import YouTube
from textwrap import wrap
from fireworks.client import Fireworks
from getpass import getpass


# Set AssemblyAI API key
aai.settings.api_key = assemblyai_api_key

# Function to download YouTube video
def download_youtube_video(youtube_url, video_output):
    yt = YouTube(youtube_url)
    video_stream = yt.streams.filter(only_audio=True).first()
    video_stream.download(filename=video_output)

# Function to extract audio from video
def extract_audio(video_path, audio_output):
    command = f"ffmpeg -i {video_path} -q:a 0 -map a {audio_output}"
    subprocess.run(command, shell=True)

# Function to upload audio to AssemblyAI
def upload_to_assemblyai(audio_path):
    headers = {
        "authorization": assemblyai_api_key,
    }
    response = requests.post("https://api.assemblyai.com/v2/upload",
                             headers=headers,
                             files={"file": open(audio_path, "rb")})
    return response.json()["upload_url"]

# Function to transcribe audio using AssemblyAI
def transcribe_audio(audio_url):
    endpoint = "https://api.assemblyai.com/v2/transcript"
    json = {
        "audio_url": audio_url
    }
    headers = {
        "authorization": assemblyai_api_key,
        "content-type": "application/json"
    }
    response = requests.post(endpoint, json=json, headers=headers)
    transcript_id = response.json()["id"]

    # Polling for the transcription result
    endpoint = f"https://api.assemblyai.com/v2/transcript/{transcript_id}"
    while True:
        response = requests.get(endpoint, headers=headers)
        result = response.json()
        if result["status"] == "completed":
            return result["text"]
        elif result["status"] == "failed":
            raise Exception("Transcription failed")
        time.sleep(5)



def get_completion(prompt, max_tokens=4096):
    fw_model_dir = "accounts/fireworks/models/"
    model = "llama-v3-70b-instruct"
    model = fw_model_dir + model
    completion = fireworks.client.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=max_tokens,
        temperature=0
    )

    return completion.choices[0].text
# Function to extract action items using Fireworks
def extract_action_items(transcription):
    prompt = f"Extract action items from the following meeting transcript:\n\n{transcription}\n\nAction items:"
    action_items = get_completion(prompt)
    return action_items.strip()


# Function to generate MCQs
def generate_mcqs(transcription):
    prompt = f"Generate 10 multiple choice questions from the following transcript. Only provide the questions and answer options without any additional text or formatting:\n\n{transcription}\n\nMCQs:"
    mcqs = get_completion(prompt)
    return mcqs.strip()

# Main process function
def process_video(video_file_path=None, youtube_url=None):
    video_file_path = "downloaded_video.mp4" if youtube_url else video_file_path
    audio_file_path = "output_audio.mp3"

    if youtube_url:
        download_youtube_video(youtube_url, video_file_path)

    extract_audio(video_file_path, audio_file_path)
    audio_url = upload_to_assemblyai(audio_file_path)
    transcription = transcribe_audio(audio_url)
    action_items = extract_action_items(transcription)
    mcqs = generate_mcqs(transcription)

    if youtube_url:
        os.remove(video_file_path)  # Clean up video file if it was downloaded
    os.remove(audio_file_path)  # Clean up audio file

    return transcription, action_items, mcqs

# Example usage
video_fileyoutube_url = input("Enter YouTube URL (leave blank if uploading a video file): ").strip()
_path = None

if not youtube_url:
    video_file_path = input("Enter the path to your video file: ").strip()

# Process the video
transcription, action_items, mcqs = process_video(video_file_path=video_file_path, youtube_url=youtube_url)

# Display the results
# wrapped_transcription = "\n".join(wrap(transcription, width=80))
# print("### Transcription:\n")
# print(wrapped_transcription)
# print("\n### Action Items:\n")
# print(action_items)
# print("\n### MCQs:\n")
# print(mcqs)
mcqs_list = [q.strip() for q in mcqs.split('\n') if q.strip()]
mcqs_dict = {}
question = ""
for i, line in enumerate(mcqs_list):
    if i % 6 == 0:
        question = line
        mcqs_dict[question] = []
    else:
        mcqs_dict[question].append(line)

# Display the MCQs dictionary
print("### MCQs Dictionary:\n")
print(mcqs_dict)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

### MCQs Dictionary:

{"1. What is the main topic of the speaker's talk?": ['a) Agents in Lingchain', 'b) React, a prompting strategy', 'c) Tool usage in language models', 'd) Evaluation of language models', 'Answer: a) Agents in Lingchain'], '2. What is the core idea of agents, according to the speaker?': ['a) Using language models as a reasoning engine', 'b) Connecting language models to search engines', 'c) Using tools to overcome limitations of language models', 'd) Improving the reliability of language models', 'Answer: a) Using language models as a reasoning engine'], '3. What is the main difference between react and auto GPT, according to the speaker?': ['a) React is designed for short-term goals, while auto GPT is designed for long-term goals', 'b) React uses tools, while auto GPT does not', 'c) React is a prompting strategy, while auto GPT is an agent framework', 'd) React is designed for language models, while auto GPT is designed for humans', 'Answer: a) React is designed fo